In [9]:
import pandas as pd
import json

In [10]:
file_name = "dataset.csv"
df = pd.read_csv(file_name)
df = df[df['book_name'] == '2023년도_지역특화사업_이야기_-_통영_한산마을_~_함안_여항면_다랑논_~_산청_단계마을'] # 책 이름만 지정
#df = df[(df['book_name'] == '비피랩_카탈로그') & (df["page_number"] == 1)] # 책 이름 + 페이지 지정
df

,book_name,page_number,1,2,3,4,5,6,7,8,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
783,2023년도_지역특화사업_이야기_-_통영_한산마을_~_함안_여항면_다랑논_~_산청_...,47.0,container,top2down,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
784,2023년도_지역특화사업_이야기_-_통영_한산마을_~_함안_여항면_다랑논_~_산청_...,47.0,NaN,icon,https://d1bzdv1wm9phyk.cloudfront.net/local/ep...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
785,2023년도_지역특화사업_이야기_-_통영_한산마을_~_함안_여항면_다랑논_~_산청_...,47.0,NaN,textbox,top2down,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
786,2023년도_지역특화사업_이야기_-_통영_한산마을_~_함안_여항면_다랑논_~_산청_...,47.0,NaN,NaN,text,“여기 단계는 붉은 단(丹)과 시내 계(溪)를 사용한다. 암석이 붉은 색을 띠고 말...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
787,2023년도_지역특화사업_이야기_-_통영_한산마을_~_함안_여항면_다랑논_~_산청_...,47.0,NaN,image,https://d1bzdv1wm9phyk.cloudfront.net/local/ep...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
788,2023년도_지역특화사업_이야기_-_통영_한산마을_~_함안_여항면_다랑논_~_산청_...,47.0,NaN,textbox,top2down,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
789,2023년도_지역특화사업_이야기_-_통영_한산마을_~_함안_여항면_다랑논_~_산청_...,47.0,NaN,NaN,text,단계딸기가 유명하다는 소문은 들었지만 실지로 맛을 보니 다른 지역보다 단맛이 더 나...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
class LayoutParser:
    def __init__(self):
        self.elements = {}
        self.element_counter = {}
    
    def generate_element_id(self, element_type):
        if element_type not in self.element_counter:
            self.element_counter[element_type] = 1
        else:
            self.element_counter[element_type] += 1
        return f"{element_type}{self.element_counter[element_type]:03d}"

    def create_layout_tree(self, df):
        ELEMENTS = ['text', 'image', 'icon']
        CONTAINERS = {
            'container': ['top2down', 'left2right'],
            'textbox': ['top2down', 'left2right'],
            'layerlist_tab': ['top', 'down'],
            'layerlist_arccodion': ['top2down', 'left2right'],
            'layerlist_slider': ['top2down', 'left2right'],
            'nac_title': ['top2down', 'left2right'],
            'nac_item': ['top2down', 'left2right'],
        }
        
        def create_container_node(symbol_name, direction):
            return {
                "symbol_name": symbol_name,
                "direction": direction,
                "children": []
            }
        
        root = create_container_node("root", "top2down")
        stack = [(root, -1)]  # (node, depth) pairs
        
        for row in df.itertuples():
            row_data = list(row)[3:]
            
            # Find depth
            depth = 0
            for val in row_data:
                if pd.isna(val):
                    depth += 1
                else:
                    break
            
            symbol_name = row_data[depth]
            
            # Find content
            content = None
            for i in range(depth + 1, len(row_data)):
                if not pd.isna(row_data[i]) and row_data[i] not in CONTAINERS and \
                   row_data[i] not in ['top2down', 'left2right', 'top', 'down']:
                    if isinstance(row_data[i], str) and not row_data[i] in ELEMENTS:
                        content = row_data[i]
                        break
            
            # Create node
            if symbol_name in CONTAINERS:
                direction = row_data[depth + 1] if len(row_data) > depth + 1 and \
                          not pd.isna(row_data[depth + 1]) else "top2down"
                new_node = create_container_node(symbol_name, direction)
            elif symbol_name in ELEMENTS:
                # Generate unique ID for element
                element_id = self.generate_element_id(symbol_name)
                # Store element in elements dictionary
                self.elements[element_id] = {
                    "type": symbol_name,
                    "content": content
                }
                # Use element_id as reference in tree
                new_node = element_id
            else:
                continue
            
            # Find appropriate parent and add node
            while stack and stack[-1][1] >= depth:
                stack.pop()
            
            if stack:
                if isinstance(new_node, dict):  # If container
                    stack[-1][0]["children"].append(new_node)
                    stack.append((new_node, depth))
                else:  # If element (string ID)
                    stack[-1][0]["children"].append(new_node)
        
        return {
            "elements": self.elements,
            "containers": root["children"][0] if root["children"] else {}
        }

In [12]:
parser = LayoutParser()
result = parser.create_layout_tree(df)

with open("output/manuscript.json", "w", encoding="utf-8") as json_file:
    json.dump(result['elements'], json_file, ensure_ascii=False, indent=4)
with open("output/symbolic_tree.json", "w", encoding="utf-8") as json_file:
    json.dump(result['containers'], json_file, ensure_ascii=False, indent=4)